# Multi-Asset Polygon Dataset: Descriptive Statistics

This notebook reproduces the **same preprocessing pipeline** as `Datasets/multi_asset_dataset.py` and then computes descriptive statistics. It uses the Polygon OHLCV parquet files and the engineered features that are fed into training (rolling z-scored returns, ranges, and volatility features).


## Configuration
Update these settings to match your training configuration (e.g., `train_jepa_initial.py`).


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd

# Dataset args that match the training
root_path = Path("Data/polygon")
data_path = Path("data_raw_1m")
start_date = None
regular_hours_only = True
timeframe = "15min" # resampling timeframe

# data args that match the training
rolling_window = 252
train_split = 0.7
test_split = 0.15

tickers = None # optional
file_suffix = ".parquet"


: 

## Helper functions (mirrors the dataset code)


In [2]:
import os


def to_datetime(series):
    try:
        return pd.to_datetime(series, unit="ms", utc=True)
    except Exception:
        return pd.to_datetime(series, utc=True)


def true_range(high, low, close):
    prev_close = close.shift(1)
    a = (high - low).abs()
    b = (high - prev_close).abs()
    c = (low - prev_close).abs()
    return pd.concat([a, b, c], axis=1).max(axis=1)


def filter_regular_us_hours(dt: pd.Series) -> pd.Series:
    dt_et = dt.dt.tz_convert("America/New_York")
    minutes = dt_et.dt.hour * 60 + dt_et.dt.minute
    is_weekday = dt_et.dt.weekday < 5
    in_rth = (minutes >= 570) & (minutes < 960)
    return is_weekday & in_rth


def resample_ohlcv(df: pd.DataFrame, rule: str) -> pd.DataFrame:
    df = df.set_index("_dt")
    agg = {
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
        "volume": "sum",
    }
    out = df.resample(rule).agg(agg).dropna()
    return out.reset_index()


def rolling_zscore(series: pd.Series, window: int) -> pd.Series:
    m = series.rolling(window, min_periods=window).mean().shift(1)
    s = series.rolling(window, min_periods=window).std().shift(1)
    return (series - m) / s


def rolling_zscore_df(df: pd.DataFrame, window: int) -> pd.DataFrame:
    out = pd.DataFrame(index=df.index, columns=df.columns, dtype=float)
    for c in df.columns:
        out[c] = rolling_zscore(df[c], window)
    return out


def iter_asset_files(path: Path, tickers=None, suffix=".parquet"):
    if tickers:
        for t in tickers:
            yield t, path / f"{t}{suffix}"
    else:
        for fname in os.listdir(path):
            if fname.endswith(suffix):
                asset_id = Path(fname).stem
                yield asset_id, path / fname


## Load assets and compute features
This step follows the exact feature engineering used for training: log returns, range/volatility features, and rolling z-scoring.


In [3]:
feature_columns = [
    "ret_close", "ret_open", "ret_high", "ret_low",
    "hl_range", "tr", "vol20", "log_vol", "vol_z20",
]


def load_asset(fpath: Path):
    df = pd.read_parquet(fpath)
    time_col = "timestamp"

    if start_date is not None:
        start_dt = pd.to_datetime(start_date, utc=True)
        dt = to_datetime(df[time_col])
        df = df.loc[dt >= start_dt].copy()

    dt = to_datetime(df[time_col])
    if regular_hours_only:
        mask = filter_regular_us_hours(dt)
        df = df.loc[mask].copy()
        dt = dt.loc[mask]

    df = df.assign(_dt=dt).sort_values("_dt", ascending=True).reset_index(drop=True)

    if timeframe != "1min":
        df = resample_ohlcv(df, timeframe)

    for col in ["open", "high", "low", "close"]:
        df[f"log_{col}"] = np.log(df[col].astype(float))

    df["ret_close"] = df["log_close"].diff()
    df["ret_open"] = df["log_open"].diff()
    df["ret_high"] = df["log_high"].diff()
    df["ret_low"] = df["log_low"].diff()

    df["hl_range"] = (df["high"] - df["low"]) / df["close"].replace(0, np.nan)
    df["tr"] = true_range(df["high"], df["low"], df["close"]) / df["close"].replace(0, np.nan)
    df["vol20"] = df["ret_close"].rolling(20, min_periods=20).std()

    df["log_vol"] = np.log(df["volume"].astype(float) + 1.0)
    df["vol_z20"] = (
        df["volume"] - df["volume"].rolling(20, min_periods=20).mean()
    ) / df["volume"].rolling(20, min_periods=20).std()

    base = df[feature_columns].copy().replace([np.inf, -np.inf], np.nan)
    base_valid = base.dropna()
    valid_index = base_valid.index

    Z = rolling_zscore_df(base.loc[valid_index], rolling_window)
    Z = Z.replace([np.inf, -np.inf], np.nan).dropna()
    final_index = Z.index

    base_final = base.loc[final_index].reset_index(drop=True)
    z_final = Z.reset_index(drop=True)

    raw_ohlcv = df.loc[final_index, ["open", "high", "low", "close", "volume"]].reset_index(drop=True)
    dt_final = df.loc[final_index, "_dt"].reset_index(drop=True)

    return {
        "base": base_final,
        "z": z_final,
        "ohlcv": raw_ohlcv,
        "dates": dt_final,
    }


data_dir = root_path / data_path
assets = []

for asset_id, fpath in iter_asset_files(data_dir, tickers=tickers, suffix=file_suffix):
    if not fpath.exists():
        continue
    data = load_asset(fpath)
    if len(data["z"]) == 0:
        continue
    assets.append({"asset": asset_id, **data})

print(f"Loaded {len(assets)} assets from {data_dir}")


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data\\polygon\\data_raw_1m'

## Data coverage summary
Summarize date ranges and row counts after preprocessing (aligned to the training features).


In [ ]:
coverage_rows = []
for item in assets:
    dates = item["dates"]
    coverage_rows.append({
        "asset": item["asset"],
        "rows": len(dates),
        "start": dates.min(),
        "end": dates.max(),
    })

coverage = pd.DataFrame(coverage_rows).sort_values("asset")
coverage


## Train/val/test split boundaries
Uses the **global date alignment** logic from `Dataset_Finance_MultiAsset`.


In [ ]:
all_dates = pd.Index(pd.concat([a["dates"] for a in assets]).unique()).sort_values()
num_total = len(all_dates)
num_train = int(num_total * train_split)
num_test = int(num_total * test_split)
num_val = num_total - num_train - num_test

train_end = all_dates[max(num_train - 1, 0)] if num_total else None
val_end = all_dates[max(num_train + num_val - 1, 0)] if num_total else None

split_info = {
    "global_total_rows": num_total,
    "train_end": train_end,
    "val_end": val_end,
    "num_train_dates": num_train,
    "num_val_dates": num_val,
    "num_test_dates": num_test,
}

split_info


## Split counts per asset


In [ ]:
split_rows = []
for item in assets:
    dates = item["dates"]
    if train_end is None:
        train_rows = val_rows = test_rows = 0
    else:
        train_rows = (dates <= train_end).sum()
        val_rows = ((dates > train_end) & (dates <= val_end)).sum()
        test_rows = (dates > val_end).sum()
    split_rows.append({
        "asset": item["asset"],
        "train_rows": int(train_rows),
        "val_rows": int(val_rows),
        "test_rows": int(test_rows),
    })

split_counts = pd.DataFrame(split_rows).sort_values("asset")
split_counts


## Descriptive statistics (training features)
These are **z-scored features** used by the model.


In [ ]:
percentiles = [0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]

per_asset_stats = []
for item in assets:
    stats = item["z"].describe(percentiles=percentiles).T
    stats["asset"] = item["asset"]
    stats["feature"] = stats.index
    per_asset_stats.append(stats.reset_index(drop=True))

z_feature_stats = pd.concat(per_asset_stats, ignore_index=True)
z_feature_stats.head()


### Global statistics across assets (z-scored features)


In [ ]:
if assets:
    z_global = pd.concat([a["z"] for a in assets], ignore_index=True)
    z_global_stats = z_global.describe(percentiles=percentiles).T
else:
    z_global_stats = pd.DataFrame()

z_global_stats


## Descriptive statistics (pre-zscore engineered features)
This table is useful to report the **raw engineered signals** before normalization.


In [ ]:
per_asset_base_stats = []
for item in assets:
    stats = item["base"].describe(percentiles=percentiles).T
    stats["asset"] = item["asset"]
    stats["feature"] = stats.index
    per_asset_base_stats.append(stats.reset_index(drop=True))

base_feature_stats = pd.concat(per_asset_base_stats, ignore_index=True)
base_feature_stats.head()


### Global statistics across assets (engineered features)


In [ ]:
if assets:
    base_global = pd.concat([a["base"] for a in assets], ignore_index=True)
    base_global_stats = base_global.describe(percentiles=percentiles).T
else:
    base_global_stats = pd.DataFrame()

base_global_stats


## Raw OHLCV descriptive statistics
Useful for reporting price/volume scales after filtering and resampling.


In [ ]:
per_asset_ohlcv_stats = []
for item in assets:
    stats = item["ohlcv"].describe(percentiles=percentiles).T
    stats["asset"] = item["asset"]
    stats["field"] = stats.index
    per_asset_ohlcv_stats.append(stats.reset_index(drop=True))

ohlcv_stats = pd.concat(per_asset_ohlcv_stats, ignore_index=True)
ohlcv_stats.head()


### Global statistics across assets (raw OHLCV)


In [ ]:
if assets:
    ohlcv_global = pd.concat([a["ohlcv"] for a in assets], ignore_index=True)
    ohlcv_global_stats = ohlcv_global.describe(percentiles=percentiles).T
else:
    ohlcv_global_stats = pd.DataFrame()

ohlcv_global_stats


## Optional: export tables for the paper


In [ ]:
output_dir = Path("notebooks/outputs")
output_dir.mkdir(parents=True, exist_ok=True)

coverage.to_csv(output_dir / "coverage_summary.csv", index=False)
split_counts.to_csv(output_dir / "split_counts.csv", index=False)
z_feature_stats.to_csv(output_dir / "z_feature_stats_by_asset.csv", index=False)
base_feature_stats.to_csv(output_dir / "base_feature_stats_by_asset.csv", index=False)
ohlcv_stats.to_csv(output_dir / "ohlcv_stats_by_asset.csv", index=False)

z_global_stats.to_csv(output_dir / "z_feature_stats_global.csv")
base_global_stats.to_csv(output_dir / "base_feature_stats_global.csv")
ohlcv_global_stats.to_csv(output_dir / "ohlcv_stats_global.csv")

output_dir
